In [ ]:
!pip install '/content/pyGPB-0.0.1.tar.gz'

Processing ./pyGPB-0.0.1.tar.gz
  Created wheel for pyGPB: filename=pyGPB-0.0.1-py3-none-any.whl size=11720 sha256=5d31656310a406a2cd3611d481970c73038bedf210caa51aa6893198601cb9fe
  Stored in directory: /root/.cache/pip/wheels/18/9e/73/f0be66f2d221d99a8ecad80a5f64821626f2a6c13e8f039eab
Successfully built pyGPB


In [ ]:
from pyGPB import LFGPB

In [ ]:
from scipy.stats import multivariate_normal as mvn
from scipy.stats import norm
from scipy.special import ndtri as norm_ppf, ndtr as norm_cdf
from scipy.linalg import solve
from scipy.integrate import quad, quad_vec

from math import sqrt, pi, exp, erf

from itertools import combinations, chain, product
import numpy as np
import pickle
import numba
import timeit

## Common Functions

In [ ]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

## Custom Multivariate CDF Computation

In [ ]:
@numba.vectorize(["float64(float64)"])
def vec_cdf(z):
    # vectorized version of the standard normal cdf
    return 0.5 * (1+erf(z/1.4142135623730950))

Implements the following result for the multivariate normal cdf where the correlation matrix is $\rho$ everywhere (expect 1s on the diagonal)
$\mathsf{F}(\mathcal{S}, \rho) = \int_{-\infty}^{\infty} \phi(u) \prod_{i\in\mathcal{S}} \Phi\left(\frac{\Phi^{-1}(p_i)-u\sqrt{\rho}}{\sqrt{1-\rho}}\right) \mathrm{d}{u}$

In [ ]:
@numba.cfunc("float64(float64, float64[:], float64)")
def integrand(u, z_values, rho):
    # Integrand of the above formual
    return vec_cdf((z_values-sqrt(rho)*u)/sqrt(1-rho)).prod() * exp(-u**2/2)/2.5066282746310005

In [ ]:
def multi_norm_cdf_vec_all_rho(z_values_vec, rho):
    # Numerical Integration of above formula
    return np.array([quad(integrand,-np.inf, np.inf, args=(z_values, rho))[0] for z_values in np.atleast_2d(z_values_vec)])

## Linear Algebra Approach

Enumerates all outcomes and aggregates, similar to naive brute-force approach to the GPB, but requires extra maths to calculate the outcome probabilities. They are acheived by solving the following equation
$\mathbf{A} \overset{o}{\boldsymbol{p}} = \overset{e}{\boldsymbol{p}}$

$	\begin{bmatrix} 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
		 & 1 & 0 & 0 & 1 & 1 & 0 & 1 \\
		 &  & 1 & 0 & 1 & 0 & 1 & 1 \\
		 &  &  & 1 & 0 & 1 & 1 & 1 \\
		 &  &  &  & 1 & 0 & 0 & 1 \\
		 &  &  &  &  & 1 & 0 & 1 \\
		 &  &  &  &  &  & 1 & 1 \\
		 &  &  &  &  &  &  & 1
	 \end{bmatrix}	
	\begin{bmatrix}op_\emptyset	\\ op_0	\\ op_1	\\ op_2	\\ op_{01} \\ op_{02} \\ op_{12} \\ op_{012} \end{bmatrix}= 
	\begin{bmatrix}ep_\emptyset	\\ ep_0	\\ ep_1	\\ ep_2	\\ ep_{01} \\ ep_{02} \\ ep_{12} \\ ep_{012} \end{bmatrix}
$

$		A_{i,j} = \begin{cases} 
			1 & \text{if } \mathsf{Pow}(M)_i \subseteq \mathsf{Pow}(M)_j \\
			0 & \mathrm{otherwise} 
		\end{cases} $

In [ ]:
def LFGPB_linalg(probs, weights, rho):
    # Length of problem
    M = len(probs)
    
    # Powerset of integers upto M, used several times in different circumstances
    power_set = list(chain.from_iterable(combinations(range(M), r) for r in range(M+1)))

    # z values for the success probs
    z_values_for_probs = norm_ppf(probs)

    # z values for the combined success events. Each row some combination event of succeses
    z_events = np.array([[np.inf if x not in successes else z_values_for_probs[x] for x in range(M)] for successes in power_set])
    
    # event probabilities corresponding to the combinations of z_events
    event_probs = multi_norm_cdf_vec_all_rho(z_events, rho)

    # interaction matrix between events and outcomes
    interaction = np.array([[all(x in z for x in y)*1 for z in power_set] for y in power_set])
    
    # outcome scenarios
    outcome_probs = solve(interaction, event_probs)
    outcome_weights = np.array([weights[list(outcome)].sum() for outcome in powerset(range(M))])

    # support of the pmf
    support_start = outcome_weights.min()
    support_end = outcome_weights.max()
    support_vector = range(support_start,support_end+1)

    # aggregate outcomes to form pmf
    pmf = np.array([sum(p for p, w in zip(outcome_probs, outcome_weights) if w==x) for x in support_vector])

    return pmf

## Testing using Linear Algebra approach

In [ ]:
for M in range(2, 11):
    # Test for M in the range {2, 3, ..., 10}
    MAEs = []
    TAEs = []
    for test_id in range(32):
        # For each M generate 32 random examples
        probs = np.random.random(M)
        weights = np.random.randint(1,20,M)
        rho = float(np.random.random(1))

        # Implementation by quadrature
        ours = LFGPB(probs=probs, weights=weights, rho=rho).cdf_vec

        # Linear Algebra approach
        theirs = LFGPB_linalg(probs=probs, weights=weights, rho=rho).cumsum()

        # Calculate Errors
        abs_errs = np.abs(ours-theirs)
        MAE = abs_errs.max()
        TAE = abs_errs.sum()
        MAEs.append(MAE)
        TAEs.append(TAE)
        
    print(M, np.array(MAEs).mean(), np.array(MAEs).std() , np.array(TAEs).mean(), np.array(TAEs).std())

2 6.212738656863337e-14 7.364197591116918e-14 9.205905894364708e-13 8.179278786775723e-13
3 4.4811946037573436e-13 1.6132762373994808e-12 2.1292857613815913e-12 4.838982440895213e-12
4 7.504571982830215e-14 1.3378654780753717e-13 1.0302035730703563e-12 1.2031560572589675e-12
5 1.1586712085663933e-13 1.6939161753890047e-13 1.3129069161596334e-12 1.040235097018335e-12
6 6.232319144810518e-13 2.664130126380249e-12 5.614712072894387e-12 2.1410358188700128e-11
7 3.380555070088293e-09 1.8819987388105846e-08 7.436742550924762e-08 4.140407295253298e-07
8 4.3428300045738125e-11 2.405751695782582e-10 7.904524846525003e-10 4.378536474277881e-09
9 4.1777307209099526e-10 2.3091054505324817e-09 6.694060499241563e-09 3.69477765681377e-08
10 1.7407401448548935e-08 9.422785949982679e-08 4.873424608960252e-07 2.6807971290924493e-06


# Testing LFGPB Using Higher Order Moments

### Functions for Calculating moments

In [ ]:
def joint_success_probs_upto_n(probs, rho, n):
    # Generate a dictionary of the joint success probabilites of all combinations
    # Of elements upto combinations of size n (inclusive)

    # Dictionary is a mapping from a tuple of sorted indices to a probability
    M = probs.shape[0]
    
    # Trivial case of no successes
    joint_probs = {():1.0}

    # Trivial case of one success
    joint_probs.update({(i,):p for i, p in enumerate(probs)})
    z_values_for_probs = norm_ppf(probs)

    # Cases for n > 2
    for k in range(2,n+1):
        combination_index_tuples = list(combinations(range(M), k))
        z_values = z_values_for_probs[np.array(combination_index_tuples)]
        k_probs = multi_norm_cdf_vec_all_rho(z_values, rho)
        joint_probs.update({i:p for i, p in zip(combination_index_tuples, k_probs)})

    return joint_probs

$			\mathrm{Co}n(\mathcal{V}_{\boldsymbol{s}}) = \left[\prod_{i \in \boldsymbol{s}} w_i \right]  \sum_{\boldsymbol{t} \subseteq \boldsymbol{s}} \left[(-1)^{n-|{\boldsymbol{t}}|} \cdot \mathsf{F}(\mathrm{Supp}(\boldsymbol{t}), \rho)  \prod_{j \in \boldsymbol{s} / \boldsymbol{t}} p_j \right]$

In [ ]:
def co_n_central_moment(indices, probs, weights, joint_probs):
    # Compute the co-n central moment for the LFGPB distribution
    # for elements at the given indices and joint_probabilities
    # already calculated and provided
    
    n = indices.shape[0]
    prob_collection = []
    for succesful_events in powerset(indices):
        failure_events = indices.copy()
        for event in succesful_events:
            failure_events = np.delete(failure_events, np.where(failure_events==event)[0][0])
        sign = (-1)**(n-len(succesful_events))
        prob = joint_probs[tuple(sorted(set(succesful_events)))] * probs[failure_events].prod()
        prob_collection.append(sign * prob)
    return np.array(prob_collection).sum() * weights[indices].prod()

$\mathrm{\mu}_n = \sum_{\boldsymbol{s} \in \{0,1,\ldots,M-1\}^n} \mathrm{Co}n(V_{s_1}, V_{s_2}, \ldots, V_{s_n})$

In [ ]:
def nth_central_moment(probs, weights, rho, n):
    # Compute the nth central moment for the LFGPB distribution 
    # from its parameters
    M = probs.shape[0]
    joint_probs = joint_success_probs_upto_n(probs, rho, n)
    co_moments = []
    for indices in product(*(range(M) for _ in range(n))):
      co_moment = co_n_central_moment(np.array(indices), probs, weights, joint_probs)
      co_moments.append(co_moment)
    return sum(co_moments)

$\mathrm{\mu}_n = \boldsymbol{\lambda} \cdot \bigcirc_{j=1}^{n} \begin{bmatrix}0-\mu \\ 1-\mu \\ \vdots \\N-1-\mu \end{bmatrix}$

where $\boldsymbol{\lambda}$ is the LFGPB pmf_vector, $\cdot$ is the dot product and $\bigcirc$ is the iterated hadamard product

In [ ]:
def nth_central_moment_from_variable(v, n):
    support_vec = np.arange(0, v.support_end+1)
    centralised_support = support_vec - v.mean()
    return (centralised_support**n * v.pmf_vec).sum()

## Check implementation

In [ ]:
nth_central_moment(np.array([0.1,0.2,0.3]),np.array([1,2,3]),0.5,3)

6.908993094821078

In [ ]:
nth_central_moment_from_variable(LFGPB(np.array([0.1,0.2,0.3]),np.array([1,2,3]),rho=0.5),3)

6.908993094821172

## Do Testing

In [ ]:
# Test the LFGPB for larger M using moments
# As moments fall over a large range, relative error is used
#
# emn = expected nth  moment, i.e. those calculated from summing the elements
# of the n-tensor of nth central mixed moments
# amn = actual nth moment, i.e. those from the pyGPB implementation 
# Note that in both cases the 3rd and 4th moments are normalized to give 
# Skewness and Kurtosis, rather than raw central moments
#
# Coding style uses numbered variables, not arrays as indices starting 
# From 0 conflict with the "n"th moment, making things confusing

for M in sorted(set(np.logspace(np.log10(4),3, num=40, dtype=np.int32))):
    #collected relative errors
    res1, res2, res3, res4 = [], [], [], []
    re1_mean, re2_mean, re3_mean, re4_mean = 0,0,0,0
    re1_std, re2_std, re3_std, re4_std = 0,0,0,0
    for test_case in range(8):
        em1, em2, em3, em4 = 0,0,0,0
        am1, am2, am3, am4 = 0,0,0,0

        # Generate random test case
        probs = np.random.random(M)
        weights = np.random.randint(1,20,M)
        rho = float(np.random.random(1))

        # Create pyGPB object
        rv = LFGPB(probs, weights, rho=rho)

        # Calculated Actual Moments
        am1 = rv.mean()
        am2 = rv.var()
        am3 = nth_central_moment_from_variable(rv, 3) / (am2**3/2)
        am4 = nth_central_moment_from_variable(rv, 4) / (am2**2)
        
        # Calculated Expected Moments. 
        em1 = (probs * weights).sum()
        res1.append(abs((am1-em1)/em1))
        
        em2 = nth_central_moment(probs, weights, rho, 2)
        res2.append(abs((am2-em2)/em2))

        if M < 64:
            em3 = nth_central_moment(probs, weights, rho, 3)  / (em2**3/2)     
            res3.append(abs((am3-em3)/em3)) 
        if M < 32:
            em4 = nth_central_moment(probs, weights, rho, 4) / (em2**2) 
            res4.append(abs((am4-em4)/em4))
        
    re1_mean, re1_std = np.array(res1).mean(), np.array(res1).std()
    re2_mean, re2_std = np.array(res2).mean(), np.array(res2).std() 
    if res3:
        re3_mean, re3_std = np.array(res3).mean(), np.array(res3).std()          
    if res4:
        re4_mean, re4_std = np.array(res4).mean(), np.array(res4).std() 

    print(M, re1_mean, re1_std, re2_mean, re2_std, re3_mean, re3_std, re4_mean, re4_std)

4 4.917707343967928e-14 8.006480911503374e-14 3.3929707795058465e-13 2.577434999316973e-13 7.68974860781515e-13 1.150968051065316e-12 6.12708637248422e-13 1.064427620798864e-12
5 4.051058012521881e-14 4.570252288987908e-14 3.7243709252456117e-13 3.25526207074465e-13 9.014733013108552e-13 1.0468383210607158e-12 4.451511933687277e-13 2.5140367765160023e-13
6 8.390304912578772e-14 1.1354356981413378e-13 5.457124920634994e-13 4.2591210740024246e-13 6.173756931384289e-12 5.096767384253235e-12 4.0056259331820944e-13 3.0783707451010663e-13
7 1.68105247838113e-14 1.8856621922696532e-14 1.198914111338268e-13 1.0836811272917676e-13 1.8679177747718947e-13 1.6319507887373695e-13 1.169289353701659e-13 1.547701045620174e-13
8 1.6850996777867402e-14 1.8244814737234264e-14 1.856236529986009e-13 1.234423226222095e-13 1.0772557497059916e-12 2.0016024740389014e-12 3.630877674974676e-13 5.630139555368871e-13
9 3.562430868927343e-14 7.189319787611688e-14 4.807906345063411e-13 7.805804209583728e-13 5.609530